# News Source Classification

In [4]:
import pandas as pd
from pprint import pprint as pp
from collections import Counter
import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [5]:
d = pd.read_csv('/root/files/sql_result.csv')
print(d.columns)

Index(['id', 'author', 'source', 'content', 'feature', 'title', 'url'], dtype='object')


# Only need the x:content and y:source
## first check the label distribution

In [6]:
%matplotlib widget
import matplotlib as  mpl
source_cnt = d['source'].value_counts()
s_cnt = dict(source_cnt)
s_cnt = sorted(s_cnt.items(),key=lambda x:x[1],reverse=True)[:10]
sc = [i[0] for i in s_cnt]
cnt = [i[1] for i in s_cnt]
plt.bar(sc,cnt)
plt.show()

FigureCanvasNbAgg()

## only take the top 10 lables

In [8]:
content = d.content.tolist()
sc_labels_t = {s:i for i,s in enumerate(sc)}
total = len(d)
fd = d[d['source'].isin(set(sc))]
pp(f'delete {total - len(content)}')
pp(sc_labels_t)
fd.head()

'delete 0'
{'中国台湾网': 7,
 '中国新闻网': 3,
 '中国证券报?中证网': 2,
 '南方日报第01版': 6,
 '参考消息网': 4,
 '央广网': 8,
 '微博': 1,
 '新华社': 0,
 '新华网': 9,
 '环球网': 5}


,id,author,source,content,feature,title,url
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
5,89612,张怡,中国证券报?中证网,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...
7,89610,申玉彬 整理,中国证券报?中证网,沙漠雄鹰：震荡有利消化套牢筹码\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近2%...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友早评：震荡有利消化套牢筹码,http://www.cs.com.cn/gppd/201706/t20170623_533...
9,89608,吴瞬,中国证券报?中证网,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",纳入MSCI指数 A股长期配置价值提升,http://www.cs.com.cn/gppd/201706/t20170623_533...
12,89605,NaN,中国新闻网,中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41...,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",因为犯罪现场的一根头发，他坐冤狱41年后终获释,http://world.huanqiu.com/hot/2017-06/10866136....


In [9]:
content = fd.content.tolist()
sc_labels = fd.source.apply(lambda x:sc_labels_t[x]).tolist()


# tools

In [10]:
import jieba
def cut(s):
    return list(jieba.cut(s))

# Design feature by content

In [11]:
import numpy as np
content = [str(i).replace('\n','').replace('\u3000','') for i in content]

## TF-IDF

In [12]:
cut_content = [cut(i) for i in content]
words = set()
for c in cut_content: words = words | set(c)
print(len(words))
w2i = {k:v for v,k in enumerate(words)}
i2w = {v:k for k,v in w2i.items()}

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.647 seconds.
Prefix dict has been built succesfully.


216071


In [13]:
print(list(words)[:100])

['57.76', '翔变', '达二', '药品', '只知', '贝汉说', '厅团', '象形文字', '黑格尔', '近到', '复掘', '几十吨', '势', '威默', '一一对应', '十堰市', 'FA18', '鸭鸣报', '难达', '严格教育', '七八百元', '48.15', '周维松', '朗伊尔城', '仍进', '叙镑', '美溪', '中维尼', '花村', '股型', '法墨', '庞超芳', '依海', '户均', '旗操', '审慎', '扣罚', '皇天', '触键', '及场域', '何新', '组占', '李强', '病树', '偃师县', '所市', '体委', '强弱', '贪心', '应靠', '原盐', '吉莉恩', '美两大', '齐湘', '奴隶制', '俄版', '承上启下', '地球化学', '漏窗', '冠冕', '蔡俊', '市同', '克罗克', '尿毒症', '仇保兴', '无现金', '加尔诺', 'Hello', '瓦其', '打越', '镇集', '音箱', '共提', '测深', '轮动料', '25.5', '郴电', '通布图', '八米', '乙为', '偏南', '达峰', '杨宝森', '再读', '双鱼', '6.1903', '采尔', '全食', '熏蒸', 'nBondIT', '留足', '中房', '撤职', '九个', '陈岭', '求之不得', '8439.3', '陈武', '斗门', '埃武拉']


In [14]:
from collections import Counter
def calculate_term_frequency(c):
    '''
    @c : cut sentence.
    @output : tf
    '''
    cnt = Counter(c)
    tf = np.zeros((len(words)))
    for k,v in dict(cnt).items():
        tf[w2i[k]] = v / len(c)
    return tf
TF = []
for c in cut_content:
    TF.append(calculate_term_frequency(c))
TF = np.array(TF)
TF.shape

(83774, 216071)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(TF,sc_labels)
lr = LogisticRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)
print(x_train.shape,y_train.shape)

In [ ]:
print(lr.score(x_test,y_test))